In [1]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

blah = SZProp.P_of_qn_corr(SZProp.lnY,M_arr,z_arr,sigN,q_arr,Mwl,lndM)

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/orphics/orphics/tools/io.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/tools/stats.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/analysis/flatMaps.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.maps. If you do not find the function you require ther

/Users/nab/Repos/SZ_filter
size 2
[[  1.56559927e-11   2.24951260e-12   1.23043796e-12 ...,   4.48134524e-13
    4.43097541e-13   4.38601667e-13]
 [  1.59234009e-11   2.27645587e-12   1.24191532e-12 ...,   4.49501443e-13
    4.44409038e-13   4.39863314e-13]
 [  1.61957527e-11   2.30382460e-12   1.25355556e-12 ...,   4.50883336e-13
    4.45735006e-13   4.41138992e-13]
 ..., 
 [  9.30895795e-10   7.26408398e-11   2.59810750e-11 ...,   1.95734572e-12
    1.87589139e-12   1.80432610e-12]
 [  9.49950010e-10   7.39892585e-11   2.64414867e-11 ...,   1.97968987e-12
    1.89696794e-12   1.82430327e-12]
 [  9.69410770e-10   7.53633409e-11   2.69105066e-11 ...,   2.00238418e-12
    1.91837093e-12   1.84458595e-12]]
((222, 130), (222, 20), (64,), (222, 20))
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. 

/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


ValueError: operands could not be broadcast together with shapes (222,130) (4440,130) 